# Load a HAWC2 time-marching simulation

HAWC2 is an aeroelastic multibody code, and we often use it to run time-marching simulations, such as the turbine's response to turbulence or a step-wind.

HAWC2 saves the time-marching files in a tabular format in various data types. For the LAC course we use the GTSDF format, which is a space-efficient binary file with extension `.hdf5`. The lacbox includes an object called `ReadHAWC2` that can load this binary data into Python so you can analyse statistics, plot results, etc.

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from lacbox.io import ReadHAWC2
from lacbox.test import test_data_path

fname = Path(test_data_path) / 'dtu_10mw_turb.hdf5'

## Load the information in the binary file

In [ ]:
h2res = ReadHAWC2(fname)

## Examine some of the attributes

Some metadata is stored in a few different attributes. The most useful one is `.chaninfo`, which contains information on the different channels saved in the res file. The attribute is a list of 3 lists:  
1. Names. A list of names for each channel, which is based on the type of output channel selected in your htc file. The names are chosen by HAWC2, unfortunately, not really descriptive or unique.  
2. Units. The units of the channel. E.g., "s" is seconds.  
3. Description. A longer description of the channel. This is how you can identify, e.g., to which body node the channel corresponds to.

Let's examine this for the demo file.

In [ ]:
# print all (public) attributes just for info
h2res.__dict__.keys()

In [ ]:
names, units, desc = h2res.chaninfo
print('There are', len(names), 'channels in this output file.')
print(names)

We can see that there are 119 channels in this output file, but the names aren't super enlightening. Let's assume I want to plot the flapwise blade-root-moment of the 3 blades. Because I remember how HAWC2 coordinate systems work, it is probably the channels whose names are `Mx coo: blade1`, `Mx coo: blade2` and `Mx coo: blade3`, which correspond to a moment around the x-axis of the blade coordinate system. However, let's verify this and also identify the body and node by looking at the corresponding descriptions for those channels.

Note that loading the file into pdap can be useful in investigating channels which channels are which because the program shows the name, units, and description in a format that is easier to parse.

In [ ]:
# find the indices that we think correspond to flapwise blade moment
idx_blades = np.where(['Mx coo: blade' in name for name in names])[0]
print('Identified indices of blade channels are', idx_blades)

# print the corresponding description
[print(desc[i]) for i in idx_blades]

# print the coresponding units
[print(units[i]) for i in idx_blades];

The `Mbdy:blade` confirms that the moment is taken on the blade body. The `blade 1 root` text at the end is optional text that I placed in the htc file to try and label the output channel as I wanted. The `nodenr:   3` indicates that, for the DTU 10 MW, the blade root moment is actually not taken at the root of the blade, but at the 3rd node, which is actually 6 m from the root of the blade. This is a choice made by the designers of the model.

All three moments are given in kNm.

## Analyse the data

Let's quickly calculate some statistics and then plots the time series.

In [ ]:
# assign the relevant data to variables for convenience
Mx1, Mx2, Mx3 = h2res.data[:, idx_blades].T

# calculate statistics
print('Mean values:')
[print(Mx.mean()) for Mx in [Mx1, Mx2, Mx3]]
Mx_mean = h2res.data[:, idx_blades].mean(axis=1)

# plot the time series
fig, ax = plt.subplots(figsize=(12, 5))
handles = ax.plot(h2res.t, h2res.data[:, idx_blades])
l, = ax.plot(h2res.t, Mx_mean, 'k--')
labels = [f'Blade {i+1}' for i in range(3)] + ['Sum of blades']
ax.set(xlim=[100, 200],
      xlabel='Time [s]',
      ylabel='Blade root moment [kNm]')
ax.legend(handles + [l], labels);